In [1]:
import grpc
import sys
import inference_pb2_grpc as pb2_grpc
import inference_pb2 as pb2

In [5]:
# Compile proto3 file
# python3 -m grpc_tools.protoc --proto_path=. ./inference.proto --python_out=. --grpc_python_out=.

In [62]:
class TestClient(object):
    """
    Client for testing inference gRPC
    """

    def __init__(self):
        self.host = '3.139.238.241'
        #self.host = 'localhost'
        self.server_port = 5125

        # instantiate a channel
        self.channel = grpc.insecure_channel(
            '{}:{}'.format(self.host, self.server_port))

        # bind the client and the server
        self.stub = pb2_grpc.InferenceStub(self.channel)

    def send(self, tx, modelHash, modelInput):
        """
        Client function to call the rpc for inference
        """
        message = pb2.InferenceParameters(tx=tx, modelHash=modelHash, modelInput = modelInput)
        print(f'{message}')
        return self.stub.RunInference(message)
    
    def pipeline(self, tx, seed, pipelineName, modelHash, modelInput):
        """
        Client function to call the rpc for inference
        """
        message = pb2.PipelineParameters(tx=tx, seed=seed, pipelineName=pipelineName, modelHash=modelHash, modelInput = modelInput)
        return self.stub.RunPipeline(message)

def testLinearModel():
    client = TestClient()
    result = client.send(tx = "0x123", modelHash="LinearTest", modelInput="[[1.0, 2.0, 3.0], [2.5, 3.5, 4.5]]")
    print("Querying " + client.host + ":" + str(client.server_port) + "...")
    print(f'{result}')

def testVolatilityModel():
    client = TestClient()
    result = client.send(tx = "0x123", modelHash="Volatility.onnx", modelInput="[[0.03],[0.05],[0.04056685],[0.03235871],[0.05629578]]")
    print("Querying " + client.host + ":" + str(client.server_port) + "...")
    print(f'{result}')
    
def testXGBSpreadModel():
    client = TestClient()
    result = client.send(tx = "0x123", modelHash="XGBSpreadModel", modelInput="[[0.003], [0.005], [0.004056685]]")
    print("Querying " + client.host + ":" + str(client.server_port) + "...")
    print(f'{result}')

def testLassoSpreadModel():
    client = TestClient()
    result = client.send(tx = "0x123", modelHash="QmXQpupTphRTeXJMEz3BCt9YUF6kikcqExxPdcVoL1BBhy", modelInput="[[0.003, 0.005, 0.004056685]]")
    print("Querying " + client.host + ":" + str(client.server_port) + "...")
    print(f'{result}')
    
def testPipeline():
    client = TestClient()
    result = client.pipeline(tx = "0x123", seed="2", pipelineName="text-generation", modelHash="gpt2-xl", modelInput="Please write me a poem about crypto?")
    print("Querying " + client.host + ":" + str(client.server_port) + "...")
    print(f'{result}')
    
def testClusteringModel():
    client = TestClient()
    result = client.send(tx = "0x123", modelHash="QmYrQvh6ixW1oQUK5Lehjmqk5eSsthiUXwQVFQA7BDf9yv", modelInput="[[0.001410,1.003477,0.007010]]")
    print("Querying " + client.host + ":" + str(client.server_port) + "...")
    print(f'{result}')
    
def testVolModel():
    client = TestClient()
    result = client.send(tx = "0x123", modelHash="QmcDnBi5PT223FmELECbZVTjitFUtTvaBoctDP88ESuAK4", modelInput="[[0.003, 0.005, 0.004056685],[0.001410,0.003477,0.007010]]")
    print("Querying " + client.host + ":" + str(client.server_port) + "...")
    print(f'{result}')

In [63]:
testClusteringModel()

tx: "0x123"
modelHash: "QmYrQvh6ixW1oQUK5Lehjmqk5eSsthiUXwQVFQA7BDf9yv"
modelInput: "[[0,7.314236392868128e-18,0.000005091525349575833]]"



_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "recvmsg:Operation timed out"
	debug_error_string = "UNKNOWN:Error received from peer  {created_time:"2023-10-22T11:03:30.207073-04:00", grpc_status:14, grpc_message:"recvmsg:Operation timed out"}"
>

In [56]:
testVolModel()

tx: "0x123"
modelHash: "QmcDnBi5PT223FmELECbZVTjitFUtTvaBoctDP88ESuAK4"
modelInput: "[[0.003, 0.005, 0.004056685],[0.001410,0.003477,0.007010]]"

Querying 3.139.238.241:5125...
tx: "0x123"
node: "664a198a8c4f622ebfcf2e8c40887994e79b59e3ecafe2bf3f2d3d72e1d752b7c0f0fc4ff0bd2208942cea0d613e148c92a7c17f06ec561b505d61dfa6a1e6d0"
value: "0.002743951"



In [57]:
testLassoSpreadModel()

tx: "0x123"
modelHash: "QmXQpupTphRTeXJMEz3BCt9YUF6kikcqExxPdcVoL1BBhy"
modelInput: "[[0.003, 0.005, 0.004056685]]"

Querying 3.139.238.241:5125...
tx: "0x123"
node: "c08e55da00fd932e7ba22792b650b99b520a1288882d58a505bfca53bbebb84e9d6910f61bbeb897da850d8c7171b7469fca999900e3dc6f1f0a8d17ccde912c"
value: "0.001360592"

